In [1]:
# add to path
import sys
import os
sys.path.append('/Users/jkingslake/Documents/science/pdemtools/src')
import pdemtools as pdt
import geopandas as gpd

import rioxarray as rxr
import geopandas as gpd
import numpy as np

from rioxarray.merge import merge_arrays
from xarray import DataArray
from shapely.geometry.polygon import Polygon
from geopandas.geodataframe import GeoDataFrame
import dask.array as da

import xarray as xr

#from ._utils import clip
from pdemtools._utils import clip

In [8]:
lake = gpd.read_file("/Users/jkingslake/Documents/projects/current_projects/AntPen_NSF_NERC/remote sensing/worldview/lake_volume/lake_outline_1.shp")
lake_bounds = lake.to_crs(3031).bounds.values[0]
np.round(lake_bounds)


array([-2371462.,  1224400., -2370452.,  1226262.])

In [9]:
bounds = [-2371462.,  1224400., -2370452.,  1226262.]  # a location on Flask Glacier in the Antarctic Peninsula

gdf = pdt.search(
    dataset='rema',
    bounds = lake_bounds,
)
dem_fpath = gdf.href_dem.values[0]
bitmask_url = gdf.href_mask.values[0]
bitmask_fpath = bitmask_url
bounds = lake_bounds
pad = True

pdt.load.from_fpath(
    gdf.href_dem.values[0],
    bounds = bounds,
    bitmask_fpath = gdf.href_mask.values[0],
    lazy_load=True,
)


<xarray.DataArray (y: 932, x: 506)> Size: 2MB
dask.array<getitem, shape=(932, 506), dtype=float32, chunksize=(932, 506), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 4kB -2.371e+06 -2.371e+06 ... -2.37e+06 -2.37e+06
  * y            (y) float64 7kB 1.226e+06 1.226e+06 ... 1.224e+06 1.224e+06
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    _FillValue:     -9999.0

In [5]:
pdt.load.from_fpath(
    dem_fpath,
    bounds = lake_bounds,
    bitmask_fpath = bitmask_url,
    lazy_load=False,
)

<xarray.DataArray (y: 932, x: 506)> Size: 2MB
array([[415.96875, 415.96875, 415.96875, ..., 412.53906, 412.57812,
        412.6172 ],
       [415.95312, 415.95312, 415.95312, ..., 412.52344, 412.5703 ,
        412.6172 ],
       [415.9375 , 415.9375 , 415.9297 , ..., 412.52344, 412.5625 ,
        412.6172 ],
       ...,
       [411.47656, 411.4375 , 411.40625, ..., 433.16406, 433.1875 ,
        433.20312],
       [411.53125, 411.4922 , 411.46094, ..., 433.1797 , 433.1953 ,
        433.21094],
       [411.57812, 411.53906, 411.51562, ..., 433.1875 , 433.1953 ,
        433.21094]], shape=(932, 506), dtype=float32)
Coordinates:
  * x            (x) float64 4kB -2.371e+06 -2.371e+06 ... -2.37e+06 -2.37e+06
  * y            (y) float64 7kB 1.226e+06 1.226e+06 ... 1.224e+06 1.224e+06
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    _FillValue:     -9999.0

In [35]:
import dask.array as da

def is_loaded(obj):
    """Check if xarray object is fully loaded into memory (no dask arrays)."""
    if isinstance(obj, xr.DataArray):
        return not isinstance(obj.data, da.Array)
    elif isinstance(obj, xr.Dataset):
        return all(not isinstance(var.data, da.Array) for var in obj.data_vars.values())
    else:
        raise TypeError("Expected an xarray Dataset or DataArray")

In [6]:

# Open dataarray using rioxarray
dem = rxr.open_rasterio(dem_fpath, chunks = {})
#is_loaded(dem)

In [13]:
dem

<xarray.DataArray (band: 1, y: 932, x: 506)> Size: 2MB
dask.array<where, shape=(1, 932, 506), dtype=float32, chunksize=(1, 932, 506), chunktype=numpy.ndarray>
Coordinates:
  * band         (band) int64 8B 1
  * x            (x) float64 4kB -2.371e+06 -2.371e+06 ... -2.37e+06 -2.37e+06
  * y            (y) float64 7kB 1.226e+06 1.226e+06 ... 1.224e+06 1.224e+06
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    _FillValue:     -9999.0

In [8]:

# Convert shapely geometry to bounds
if isinstance(bounds, Polygon):
    bounds = bounds.bounds

# Clip if requested, or get whole bounds if not
if bounds is not None:
    dem = clip(dem, bounds)
else:
    bounds = dem.rio.bounds()
    if pad is not False:
        raise ValueError("If `bounds` is not provided, `pad` must be False")


In [10]:

# Filter -9999.0 values
dem = dem.where(dem > -9999.0)


In [12]:

# Mask using bitmask if requested
if bitmask_fpath is not None:
    mask = rxr.open_rasterio(bitmask_fpath)
    if bounds is not None:
        mask = clip(mask, bounds)
    dem = dem.where(mask == 0)
    del mask


In [14]:

# Remove `band` dim
dem = dem.squeeze(drop=True)

In [15]:
dem

<xarray.DataArray (y: 932, x: 506)> Size: 2MB
dask.array<getitem, shape=(932, 506), dtype=float32, chunksize=(932, 506), chunktype=numpy.ndarray>
Coordinates:
  * x            (x) float64 4kB -2.371e+06 -2.371e+06 ... -2.37e+06 -2.37e+06
  * y            (y) float64 7kB 1.226e+06 1.226e+06 ... 1.224e+06 1.224e+06
    spatial_ref  int64 8B 0
Attributes:
    AREA_OR_POINT:  Area
    scale_factor:   1.0
    add_offset:     0.0
    _FillValue:     -9999.0